In [12]:
from glob import glob
from tqdm import tqdm
from PIL import Image

import json
import ollama

In [ ]:
def pad_to_square(img, size=1024, fill_color="black"):
    width, height = img.size
    
    # Calculate new square size (max of width/height)
    new_size = max(width, height)
    
    # Create new square image with fill color
    if fill_color == 'auto':
        # Sample edge pixel for background color
        fill_color = img.getpixel((0, 0))
    
    new_img = Image.new('RGB', (new_size, new_size), fill_color)
    
    # Calculate position to paste original image (centered)
    paste_x = (new_size - width) // 2
    paste_y = (new_size - height) // 2
    
    # Paste original image onto square canvas
    new_img.paste(img, (paste_x, paste_y))
    
    return new_img.resize((size, size), Image.Resampling.LANCZOS)

def fnv1a_32(data: bytes) -> int:
    FNV_PRIME = 0x01000193      # 16777619
    FNV_OFFSET = 0x811c9dc5     # 2166136261
    
    hash_val = FNV_OFFSET
    for byte in data:
        hash_val = hash_val ^ byte
        hash_val = (hash_val * FNV_PRIME) & 0xFFFFFFFF  # Keep 32-bit
    
    return hash_val

def fnv_hash_string(text: str, encoding='utf-8') -> str:
    """Hash text and return as decimal string"""
    return str(fnv1a_32(text.encode(encoding)))

In [92]:
def ask_ollama(prompt):
    response = ollama.chat(
        model='qwen3:8b',
        messages=[
            {'role': 'system', 'content': "You are an expert dental radiologist tasked with making a diagnosis that returns structured responses in json. Responses should always be in the format {diagnosis,treatment,tooth}. The tooth key should always contain the singular most affected tooth with underscores no spaces. For example: upper_right_central_incisor. The treatment key should be one of restoration or extraction or RCT. A restoration can be referred to as a filling in the diagnosis."},
            {'role': 'user', 'content': prompt}
        ],
        options={
            'temperature': 0.1,      # Creativity (0-1)
            'num_predict': 212,      # Max tokens
            'top_p': 0.9,           # Nucleus sampling
            'seed': 42,             # Reproducibility
        },
        think=False
    )
    return response.message.content

In [93]:
with open("individual_x_ray_dataset/dataset.json") as f:
    data = json.load(f)

In [94]:
for datum in tqdm(data):
    img = Image.open(datum['file'])
    structured_response = ask_ollama(f'Give me the affected tooth, diagnosis, and treatment. Be succinct but factual in diagnosis and do not omit anything. {datum["objects"]} ')
    structured_json = json.loads(structured_response)
    new_datum = datum.copy()
    
    new_datum["objects"] = [structured_json]
    new_datum["objects"][0]["object_id"] = f"item-{fnv_hash_string(structured_json['diagnosis'])}_{structured_json['tooth']}_individual"
    new_datum["objects"][0]["wd"] = 0
    new_datum["objects"][0]["ht"] = 0
    new_datum["objects"][0]["x1"] = 0
    new_datum["objects"][0]["x2"] = 0
    new_datum["objects"][0]["y1"] = 0
    new_datum["objects"][0]["y2"] = 0

    with open("dataset/dataset.json", 'r') as f:
            all_data = json.load(f)
    all_data.append(new_datum)
    for idx,item in enumerate(all_data):
           item["id"] = idx
    with open("dataset/dataset.json", 'w') as f:
            json.dump(all_data, f, indent=2)

100%|██████████| 322/322 [11:53<00:00,  2.22s/it]


In [95]:
images = sorted(glob("dataset/*.jpg"))

In [103]:
images[0]

'dataset/extraction_img1.jpg'

In [102]:
for im in tqdm(images):
    image = pad_to_square(Image.open(im))
    image.save(im)

100%|██████████| 508/508 [00:15<00:00, 33.42it/s]
